In [15]:
#Dependencies
import os
import json 
#import wordcloud
import requests
import pandas as pd
from scipy import stats
from config import api_key
import googleapiclient.errors
import matplotlib.pyplot as plt
import googleapiclient.discovery
from youtube_api import YoutubeDataApi
from googleapiclient.discovery import build

In [16]:
#youtube, looking through youtube api
youtube = build("youtube", "v3", developerKey=api_key)

In [17]:
#use api_run_stop, False calls the API/ True skips the API 
api_run_stop = False 
if not api_run_stop:
    top50_qc = youtube.search().list(q="Quarantine Challenge",
                                     part=["snippet", "id"],
                                     type="video",
                                     publishedAfter="2020-03-14T00:00:00Z", 
                                     publishedBefore="2020-03-14T23:59:00Z",
                                     videoDuration="any", 
                                     order="viewCount", 
                                     maxResults=10)
    top50_qc2 = top50_qc.execute()
    top50_qc2

In [19]:
    top50_qc_df = pd.DataFrame(top50_qc2['items'])
    top50_qc_df.to_json('SEARCH_CACHE.json')
    top50_qc_df=pd.DataFrame(pd.read_json('SEARCH_CACHE.json')) 
print(json.dumps(top50_qc, indent=4, sort_keys=True))

TypeError: Object of type HttpRequest is not JSON serializable

In [5]:
#df_video_ids, unpacks the dictionary stored in "id" and creats a data frame
df_video_ids = pd.json_normalize(top50_qc_df['id'])

#df_snippets, unpacks the dictionary stored in "snippet" and creats a data frame
df_snippets = pd.json_normalize(top50_qc_df['snippet'])

#df_top50_merge, merges the above two data frames into one
df_top50_merge = pd.merge(df_video_ids , df_snippets, left_index=True, right_index=True)

#request, gets the 'videoId' and statistics from video api
request = youtube.videos().list(part='id,statistics,snippet',id=list(df_top50_merge['videoId']))
response = request.execute()# json

df_stat = pd.DataFrame(response['items']) # response['items'] list of dicts

df_stat_clean = pd.json_normalize(df_stat['statistics']) 

# merge df_stat_clean with df_top50_merge
df_stat_merge = pd.merge(df_top50_merge, df_stat_clean, left_index=True, right_index=True)
#qc_df_final, drop irrelevant columns to present cleaner dataframe
qc_df_final = df_stat_merge.drop(columns=['kind', 'liveBroadcastContent', 
                                          'publishTime', 'thumbnails.default.url', 'thumbnails.medium.width', 
                                          'thumbnails.medium.height', 'thumbnails.high.url', 'thumbnails.high.width', 
                                          'thumbnails.high.height', 'thumbnails.default.width', 'thumbnails.default.height', 
                                          'thumbnails.medium.url'])

qc_df_final.head()

#channels, gets the 'channelId' and statistics from youtube channels api
channels=youtube.channels().list(part='statistics',id=list(qc_df_final['channelId']), maxResults=50).execute()

#channels_df, creats a dataFrame of channels
channels_df = pd.DataFrame(channels['items'])

#channels_stat, unpack "statistics" into a dataFrame
channels_stat = pd.json_normalize(channels_df['statistics'])

#channels_merge, merge new stat with above df
channels_merge = pd.merge(qc_df_final, channels_stat, left_index=True, right_index=True)

#clean_data, rename columns for cleaner look
clean_data = channels_merge.rename(columns={"viewCount_x": "VideoViewCount", "viewCount_y": "channelViewCount", 
                                            "title": "VideoTitle", "commentCount_x": "Vid_commentCount"})
#qc_stat, displays a dataframe withcleaner order 
qc_stat = pd.DataFrame(clean_data, columns=["channelTitle", "VideoTitle", "channelViewCount", "subscriberCount", 
                                            "videoCount", "VideoViewCount", "description"])
qc_stat.head(1)
#.set_index(["channelTitle", "VideoTitle"])

KeyError: 'id'